In [5]:
import cupy as np
import matplotlib
import cv2



print("Versi NumPy (melalui CuPy):", np.__version__)
print("Versi Matplotlib:", matplotlib.__version__)
print("Versi OpenCV (cv2):", cv2.__version__)
!pip show tqdm


# Versi NumPy (melalui CuPy): 13.0.0
# Versi Matplotlib: 3.7.5
# Versi OpenCV (cv2): 4.9.0
# Name: tqdm
# Version: 4.66.1
# Summary: Fast, Extensible Progress Meter
# Home-page: 
# Author: 
# Author-email: 
# License: MPL-2.0 AND MIT
# Location: /opt/conda/lib/python3.10/site-packages


Versi NumPy (melalui CuPy): 13.0.0
Versi Matplotlib: 3.7.5
Versi OpenCV (cv2): 4.9.0
Name: tqdm
Version: 4.66.1
Summary: Fast, Extensible Progress Meter
Home-page: 
Author: 
Author-email: 
License: MPL-2.0 AND MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: 
Required-by: beatrix_jupyterlab, catalyst, cmdstanpy, conda, datasets, dipy, featuretools, fitter, google-generativeai, huggingface-hub, hyperopt, kaggle, kagglehub, lime, mlcrate, mne, momepy, optuna, panel, papermill, prophet, pytorch-lightning, scattertext, segregation, shap, spacy, spopt, tensorflow-datasets, tensorpack, tobler, torchtext, TPOT, transformers, tsfresh, umap-learn, ydata-profiling


In [6]:
import cupy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import os.path

np.random.seed(50)

CATEGORIES = ['Bacterialblight', 'Blast', 'Brownspot','Tungro']
IMG_SIZE = 32

def get_data(DATADIR, raw_training_data, raw_testing_data):
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)  # Path ke sub-direktori kategori
        class_num = CATEGORIES.index(category)

        num_files = len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))])
        count = 0

        for img in sorted(tqdm(os.listdir(path))):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                # Check if image array is not None
                if img_array is None:
                    print(f"Failed to read image: {os.path.join(path, img)}")
                    continue

                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                count += 1
                if count < num_files - 99:
                    raw_training_data.append([new_array, class_num])
                else:
                    raw_testing_data.append([new_array, class_num])
            except Exception as e:
                # Log the exception
                print(f"Error processing image {os.path.join(path, img)}: {e}")

raw_training_data = []
raw_testing_data = []

DATADIR = "rice-leaf-disease-image"

get_data(DATADIR, raw_training_data, raw_testing_data)

print(len(raw_training_data), len(raw_testing_data))


100%|██████████| 1308/1308 [00:00<00:00, 990458.50it/s]


5532 400


In [7]:
from sklearn.utils import shuffle
from typing import Tuple

X_TRAIN = []
Y_TRAIN = []

X_TEST = []
Y_TEST = []

X_VAL = []
Y_VAL = []


for features,label in raw_training_data:
    X_TRAIN.append(features)
    Y_TRAIN.append(label)


for features,label in raw_testing_data:
    X_TEST.append(features)
    Y_TEST.append(label)

X_TRAIN = np.array(X_TRAIN).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X_TEST = np.array(X_TEST).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y_TRAIN = np.array(Y_TRAIN)
X_TRAIN, Y_TRAIN = shuffle(X_TRAIN, Y_TRAIN)

X_TRAIN = X_TRAIN.astype(float) / 255.
X_TEST = X_TEST.astype(float) / 255.


X_TRAIN, X_VAL = X_TRAIN[:-500], X_TRAIN[-500:]
Y_TRAIN, Y_VAL = Y_TRAIN[:-500], Y_TRAIN[-500:]

"""
np.save('xtest.npy',X_TEST)
np.save('ytest.npy',Y_TEST)
"""

print (X_TRAIN.shape, X_VAL.shape)

(5032, 32, 32, 3) (500, 32, 32, 3)


In [8]:
class conv_layer():

    def __init__(self, total_filters, filter_shape, lrate = 0.1, padding = 'no_padding', stride = 1, beta1 = 0.9, beta2 = 0.999):
        self.filters_ws = np.random.randn(*filter_shape, total_filters) * 0.1
        self.filter_bs = np.random.randn(total_filters) * 0.1
        self.padding = padding
        self.stride = stride
        self.din_dw = None 
        self.din_db = None
        self.input = None
        self.mo = 0
        self.acc = 0
        self.mo_b = 0
        self.acc_b = 0
        self.beta1 = beta1
        self.beta2 = beta2
        self.lr = lr

    def forward(self, input, training):
        #print(self.filters_ws)
        self.input = np.array(input, copy=True)


        ############### getting output dimentions here ###############
        n, input_dim_h, input_dim_w, _ = input.shape
        filter_dim_h, filter_dim_w, _, filter_dim_n = self.filters_ws.shape
        if self.padding == 'keep_img_dim':
            output_shape = n, input_dim_h, input_dim_w, filter_dim_n
            filter_dim_h, filter_dim_w, _, _ = self.filters_ws.shape
            p_value = (filter_dim_h - 1) // 2, (filter_dim_w - 1) // 2
        elif self.padding == 'no_padding':
            out_dim_h = (input_dim_h - filter_dim_h) // self.stride + 1
            out_dim_w = (input_dim_w - filter_dim_w) // self.stride + 1
            output_shape = n, out_dim_h, out_dim_w, filter_dim_n
            p_value = 0, 0
        ############### got output dimentions ###############

        out_dim_n, out_dim_h, out_dim_w, out_dim_c = output_shape

        input_padded = self.pad(input, p_value)
        output = np.zeros(output_shape)

        for i in range(out_dim_h):
            for j in range(out_dim_w):
                start_pix_x = i * self.stride
                end_pix_x = start_pix_x + filter_dim_h
                start_pix_y = j * self.stride
                end_pix_y = start_pix_y + filter_dim_w

                output[:, i, j, :] = np.sum(
                    input_padded[:, start_pix_x:end_pix_x, start_pix_y:end_pix_y, :, np.newaxis] *
                    self.filters_ws[np.newaxis, :, :, :],
                    axis=(1, 2, 3)
                )

        #print(output)
        return output + self.filter_bs

    def backward(self, input , grad):
        out_dim_n, out_dim_h, out_dim_w, out_dim_c = grad.shape
        n, input_dim_h, input_dim_w, input_dim_c = self.input.shape
        filter_dim_h, filter_dim_w, _, _ = self.filters_ws.shape


        ############### getting p value here ###############
        if self.padding == 'keep_img_dim':
            p_value = (filter_dim_h - 1) // 2, (filter_dim_w - 1) // 2
        elif self.padding == 'no_padding':
            p_value = 0, 0
        ############### got p value  ###############



        input_padded = self.pad(self.input, p_value)
        output = np.zeros_like(input_padded)
        #print(grad)
        self.din_db = grad.sum(axis=(0, 1, 2)) / n
        self.din_dw = np.zeros_like(self.filters_ws)

        for i in range(out_dim_h):
            for j in range(out_dim_w):
                start_pix_x = i * self.stride
                end_pix_x = start_pix_x + filter_dim_h
                start_pix_y = j * self.stride
                end_pix_y = start_pix_y + filter_dim_w
                output[:, start_pix_x:end_pix_x, start_pix_y:end_pix_y, :] += np.sum(
                    self.filters_ws[np.newaxis, :, :, :, :] *
                    grad[:, i:i+1, j:j+1, np.newaxis, :],
                    axis=4
                )
                self.din_dw += np.sum(
                    input_padded[:, start_pix_x:end_pix_x, start_pix_y:end_pix_y, :, np.newaxis] *
                    grad[:, i:i+1, j:j+1, np.newaxis, :],
                    axis=0
                )

        self.din_dw /= n

        #print(self.din_dw)

        ###################### Adam ###############################
        self.mo = self.beta1*self.mo + (1-self.beta1)*(self.din_dw) 
        self.acc = self.beta2*self.acc + (1-self.beta2)*((self.din_dw) *(self.din_dw))
        self.filters_ws += -self.lr * self.mo / (np.sqrt(self.acc) + 1e-7)

        self.mo_b = self.beta1*self.mo_b + (1-self.beta1)*(self.din_db) 
        self.acc_b = self.beta2*self.acc_b + (1-self.beta2)*((self.din_db) *(self.din_db))
        self.filter_bs += -self.lr * self.mo_b / (np.sqrt(self.acc_b) + 1e-7)        
        ###################### Adam ###############################

        return output[:, p_value[0]:p_value[0]+input_dim_h, p_value[1]:p_value[1]+input_dim_w, :]


    @staticmethod
    def pad(array, pad):
        return np.pad(
            array=array,
            pad_width=((0, 0), (pad[0], pad[0]), (pad[1], pad[1]), (0, 0)),
            mode='constant'
        )



In [9]:
class pool_layer():

    def __init__(self, input_dim, stride = 2):
        self.pool_dim = input_dim
        self.stride = stride
        self.input = None
        self.max_pixels = {}

    def forward(self, input, training):
        #print(input.shape)
        self.input = np.array(input, copy=True)
        n, input_dim_h, input_dim_w, c = input.shape
        pool_x_dim, pool_y_dim = self.pool_dim
        out_dim_h = 1 + (input_dim_h - pool_x_dim) // self.stride
        out_dim_w = 1 + (input_dim_w - pool_y_dim) // self.stride
        output = np.zeros((n, out_dim_h, out_dim_w, c))

        for i in range(out_dim_h):
            for j in range(out_dim_w):
                start_pix_x = i * self.stride
                end_pix_x = start_pix_x + pool_x_dim
                start_pix_y = j * self.stride
                end_pix_y = start_pix_y + pool_y_dim
                focus_area = input[:, start_pix_x:end_pix_x, start_pix_y:end_pix_y, :]
                self.store_max_pixels(focus_area, (i, j))
                output[:, i, j, :] = np.max(focus_area, axis=(1, 2))

        #print(output)
        return output

    def backward(self, input,grad):
        #print(grad)
        output = np.zeros_like(self.input)
        n, out_dim_h, out_dim_w, c = grad.shape
        pool_x_dim, pool_y_dim = self.pool_dim
        for i in range(out_dim_h):
            for j in range(out_dim_w):
                start_pix_x = i * self.stride
                end_pix_x = start_pix_x + pool_x_dim
                start_pix_y = j * self.stride
                end_pix_y = start_pix_y + pool_y_dim
                output[:, start_pix_x:end_pix_x, start_pix_y:end_pix_y, :] += grad[:, i:i + 1, j:j + 1, :] * self.max_pixels[(i, j)]
        #print(output)
        return output

    def store_max_pixels(self, area_pixels, i_j_location):
        mark_max = np.zeros_like(area_pixels)
        n, h, w, c = area_pixels.shape
        area_pixels = area_pixels.reshape(n, h * w, c)
        max_locations = np.argmax(area_pixels, axis=1)
        n_idx, c_idx = np.indices((n, c))
        mark_max.reshape(n, h * w, c)[n_idx, max_locations, c_idx] = 1
        self.max_pixels[i_j_location] = mark_max



In [10]:
class reshape_layer():

    def __init__(self):
        self.input_shape = ()

    def forward(self, input, training):
        self.input_shape = input.shape
        return np.ravel(input).reshape(input.shape[0], -1)

    def backward(self,input , grad):
        return grad.reshape(self.input_shape)

In [11]:
all_ws = []
class weights_layer():
    def __init__(self, fan_in, fan_out, lr=0.05, beta1 = 0.9, beta2 = 0.999 , lamdaa = 0.0001):
        self.lamdaa = lamdaa
        self.lr = lr
        self.ws = np.random.randn(fan_in, fan_out)/np.sqrt(fan_in)
        self.bs = np.zeros(fan_out)
        self.mo = 0
        self.acc = 0
        self.mo_b = 0
        self.acc_b = 0
        self.beta1 = beta1
        self.beta2 = beta2
        
    def forward(self,input, training):
        #all_ws.append(self.ws)
        #print(input.shape)
        #print(np.dot(input,self.ws) + self.bs)
        return np.dot(input,self.ws) + self.bs
    
    def backward(self,input,grad_output):
        dout_din = np.dot(grad_output, self.ws.T)
        dout_dws = np.dot(input.T, grad_output)
        dout_dbs = grad_output.mean(axis=0)*input.shape[0]
        
        assert dout_dws.shape == self.ws.shape and dout_dbs.shape == self.bs.shape
        #print(dout_dws)
        ###################### Adam ###############################
        self.mo = self.beta1*self.mo + (1-self.beta1)*(dout_dws) 
        self.acc = self.beta2*self.acc + (1-self.beta2)*((dout_dws) *(dout_dws))
        self.ws += -self.lr * self.mo / (np.sqrt(self.acc) + 1e-7)

        self.mo_b = self.beta1*self.mo_b + (1-self.beta1)*(dout_dbs) 
        self.acc_b = self.beta2*self.acc_b + (1-self.beta2)*((dout_dbs) *(dout_dbs))
        self.bs += -self.lr * self.mo_b / (np.sqrt(self.acc_b) + 1e-7) 
        ###################### Adam ###############################
        """
        self.ws = self.ws - self.lr * dout_dws  #+ (self.lamdaa * np.sum(self.ws))/input.shape[0]
        self.bs = self.bs - self.lr * dout_dbs  #+ (self.lamdaa * np.sum(self.bs))/input.shape[0]  
        """
        #print(dout_din)
        return dout_din

In [12]:
class ReLU():
    def __init__(self):
        pass
    
    def forward(self, input, training):
        relu_forward = np.maximum(0,input)
        return relu_forward
    
    def backward(self, input, grad_output):
        relu_grad = input > 0
        return grad_output*relu_grad

class tanh():
    def __init__(self):
        pass
    def forward(self, input, training):
        return np.tanh(input)

    def backward(self, input, grad_output):
      return grad_output*(1-np.tanh(input)**2)

class sigmoid():
    def __init__(self):
        pass
    def forward(self, input, training):
        return 1/(1+np.exp(-1* input))
    def backward(self, input, grad_output):
        return grad_output * (input*(1-input))


In [13]:
class ReLU():
    def __init__(self):
        pass
    
    def forward(self, input, training):
        relu_forward = np.maximum(0,input)
        return relu_forward
    
    def backward(self, input, grad_output):
        relu_grad = input > 0
        return grad_output*relu_grad

class tanh():
    def __init__(self):
        pass
    def forward(self, input, training):
        return np.tanh(input)

    def backward(self, input, grad_output):
      return grad_output*(1-np.tanh(input)**2)

class sigmoid():
    def __init__(self):
        pass
    def forward(self, input, training):
        return 1/(1+np.exp(-1* input))
    def backward(self, input, grad_output):
        return grad_output * (input*(1-input))


In [14]:
def NLL(expected_probabilities,actual_labels, m, lamdaa = 0.0001):


    correct_prob = expected_probabilities[np.arange(len(expected_probabilities)),actual_labels]

    p = np.exp(correct_prob) / np.sum(np.exp(expected_probabilities),axis=-1)

    loss = -1 * np.log(p)
    
    """
    s_reg = 0
    for i in all_ws:
      s_reg += np.sum(np.square(i))
    """
    

    #print(expected_probabilities)
    return loss #+ (lamdaa * s_reg)/m

def back_NLL(expected_probabilities,actual_labels):

    hotmap = np.zeros_like(expected_probabilities)
    hotmap[np.arange(len(expected_probabilities)),actual_labels] = 1
    
    ratios = np.exp(expected_probabilities) / np.exp(expected_probabilities).sum(axis=-1,keepdims=True)
    
    
    #print((- hotmap + ratios) / expected_probabilities.shape[0]) 
    return (- hotmap + ratios) / expected_probabilities.shape[0]


In [15]:
def run_batch(CNN, X, training):
    all_layers_outputs = []
    received = X
    for layer in CNN:
        all_layers_outputs.append(layer.forward(received, training))
        received = all_layers_outputs[-1]
  
    all_ws=[]
    assert len(all_layers_outputs) == len(CNN)
    
    return all_layers_outputs

def predict(CNN,X, Y, training):
    expected_probabilities = run_batch(CNN,X, training)[-1]
    losses = NLL(expected_probabilities,Y, X.shape[0])
    return (expected_probabilities.argmax(axis=-1) , np.mean(losses))

def train(CNN,X,acutal_labels, training):

    layers_outputs = run_batch(CNN,X, training)
    layers_inputs = [X]+layers_outputs 
    expected_probs = layers_outputs[-1]
    loss = NLL(expected_probs,acutal_labels, X.shape[0])
    loss_grad = back_NLL(expected_probs,acutal_labels)

    for layer_index in range(len(CNN))[::-1]:
        layer = CNN[layer_index]
        loss_grad = layer.backward(layers_inputs[layer_index],loss_grad)
        
    return np.mean(loss)

In [16]:
class dropout_layer():

    def __init__(self, keep_prob):
        self.cutoff_prob = keep_prob
        self.zeros_for_dropped = None

    def forward(self, input, training):
        if training:
            self.zeros_for_dropped = (np.random.rand(*input.shape) < self.cutoff_prob)
            return self.drop(input, self.zeros_for_dropped)
        else:
            return input

    def backward(self, input, grad):
        return self.drop(grad, self.zeros_for_dropped)

    def drop(self, input, zeros_for_dropped):
        input *= zeros_for_dropped
        input /= self.cutoff_prob
        #print(input)
        return input

In [17]:
from tqdm import trange
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.random.permutation(len(inputs))
    for start_idx in trange(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [18]:
# max_count = 20
# learning_rates = []
# train_log = []
# val_log = []
# train_losses = []
# val_losses = []
# n_epochs = 3

# for count in range(max_count):

#   lr = 10**np.random.uniform(-0.5,-4)

#   CNN = []

#   CNN.append(conv_layer(8, (3,3,3), lr))
#   CNN.append(ReLU())
#   CNN.append(pool_layer((2,2)))

#   CNN.append(conv_layer(16, (3,3,8) , lr))
#   CNN.append(ReLU())
#   CNN.append(pool_layer((2,2)))


#   CNN.append(conv_layer(32, (3,3,16), lr))
#   CNN.append(ReLU())
#   CNN.append(pool_layer((2,2)))


#   CNN.append(reshape_layer())


#   CNN.append(weights_layer(128,100,lr))
#   CNN.append(ReLU())
#   CNN.append(dropout_layer(0.6))
#   CNN.append(weights_layer(100,100,lr))
#   CNN.append(ReLU())
#   CNN.append(dropout_layer(0.6))
#   CNN.append(weights_layer(100,5,lr))

#   for epoch in range(n_epochs):

#     train(CNN,X_TRAIN,Y_TRAIN, 1)

#     train_predictions, train_loss = predict(CNN,X_TRAIN, Y_TRAIN, 0)

#     val_predictions, val_loss = predict(CNN,X_VAL, Y_VAL, 0)

#     train_log.append(np.mean(train_predictions==Y_TRAIN))

#     val_log.append(np.mean(val_predictions==Y_VAL))

#     train_losses.append(train_loss)

#     val_losses.append(val_loss)

#     learning_rates.append(lr)

#     print("Trial %d epoch %d got t_acc = %f, v_acc = %f, t_loss = %f, and v_loss = %f at lr = %f" % (count, epoch, train_log[-1], train_losses[-1],val_losses[-1], val_log[-1], lr))

# print()
# print("Got highest train accuracy = %f at lr = %f" % (max(train_log), learning_rates[train_log.index(max(train_log))]))
# print("Got highest val accuracy = %f at lr = %f" % (max(val_log), learning_rates[val_log.index(max(val_log))]))
# print("Got lowest train loss = %f at lr = %f" % (min(train_losses), learning_rates[train_losses.index(min(train_losses))]))
# print("Got lowest val loss = %f at lr = %f" % (min(val_losses), learning_rates[val_losses.index(min(val_losses))]))


In [19]:
lr=0.0007

train_log = []
val_log = []
losses = []
train_count = []
counter = 0
train_losses = []
val_losses = []

CNN = []

CNN.append(conv_layer(8, (3,3,3), lr))
CNN.append(ReLU())
CNN.append(pool_layer((2,2)))

CNN.append(conv_layer(16, (3,3,8) , lr))
CNN.append(ReLU())
CNN.append(pool_layer((2,2)))


CNN.append(conv_layer(32, (3,3,16), lr))
CNN.append(ReLU())
CNN.append(pool_layer((2,2)))


CNN.append(reshape_layer())


CNN.append(weights_layer(128,100,lr))
CNN.append(ReLU())
CNN.append(dropout_layer(0.5))
CNN.append(weights_layer(100,100,lr))
CNN.append(ReLU())
CNN.append(dropout_layer(0.5))
CNN.append(weights_layer(100,5,lr))

n_epochs = 200

for epoch in range(n_epochs):
    
    #for x_batch,y_batch in iterate_minibatches(X_TRAIN,Y_TRAIN,batchsize=534,shuffle=True):
    #  train(CNN,x_batch,y_batch, 1)

    train(CNN,X_TRAIN,Y_TRAIN,1)

    train_predictions, train_loss = predict(CNN,X_TRAIN, Y_TRAIN, 0)

    val_predictions, val_loss = predict(CNN,X_VAL, Y_VAL, 0)

    train_log.append(np.mean(train_predictions==Y_TRAIN))

    val_log.append(np.mean(val_predictions==Y_VAL))

    train_losses.append(train_loss)

    val_losses.append(val_loss)

    print("Ran epoch %d and got t_acc = %f val_acc = %f, t_loss = %f, v_loss = %f" % (epoch, train_log[-1], val_log[-1], train_losses[-1], val_losses[-1]))



print("For Q4:")
print("Best train accuracy:",max(train_log))
print("Best val accuracy:",max(val_log))



Ran epoch 0 and got t_acc = 0.259141 val_acc = 0.304000, t_loss = 1.590091, v_loss = 1.588718
Ran epoch 1 and got t_acc = 0.281598 val_acc = 0.276000, t_loss = 1.549434, v_loss = 1.549227
Ran epoch 2 and got t_acc = 0.297099 val_acc = 0.296000, t_loss = 1.508914, v_loss = 1.509761
Ran epoch 3 and got t_acc = 0.413951 val_acc = 0.410000, t_loss = 1.463443, v_loss = 1.466126
Ran epoch 4 and got t_acc = 0.299086 val_acc = 0.266000, t_loss = 1.423076, v_loss = 1.429361
Ran epoch 5 and got t_acc = 0.270270 val_acc = 0.248000, t_loss = 1.396303, v_loss = 1.406475
Ran epoch 6 and got t_acc = 0.270270 val_acc = 0.248000, t_loss = 1.384611, v_loss = 1.395982
Ran epoch 7 and got t_acc = 0.270270 val_acc = 0.248000, t_loss = 1.379468, v_loss = 1.387623
Ran epoch 8 and got t_acc = 0.270270 val_acc = 0.248000, t_loss = 1.378342, v_loss = 1.382060
Ran epoch 9 and got t_acc = 0.273052 val_acc = 0.248000, t_loss = 1.380376, v_loss = 1.381370
Ran epoch 10 and got t_acc = 0.463037 val_acc = 0.438000, t_

In [20]:
num_partition = 4
x_test_arrays = np.split(X_TEST, num_partition)
y_test_arrays = np.split(np.array(Y_TEST), num_partition)
n_images = len (x_test_arrays[0])

accuracies = []

print ('For Q5:')
for i in range(num_partition):
    class_predictions, class_loss = predict(CNN,x_test_arrays[i], y_test_arrays[i], 0)
    accuracy = np.mean(class_predictions==y_test_arrays[i])
    print('For Category ' , CATEGORIES[i],' , Got Accuracy' , accuracy)
    accuracies.append(accuracy)

y_pos = np.arange(len(CATEGORIES))





For Q5:
For Category  Bacterialblight  , Got Accuracy 0.84
For Category  Blast  , Got Accuracy 0.71
For Category  Brownspot  , Got Accuracy 0.95
For Category  Tungro  , Got Accuracy 0.98


In [21]:
predicted_test, test_loss = predict(CNN,X_TEST, Y_TEST, 0)
accuracy = np.mean(predicted_test==np.array(Y_TEST))
    
print("َََQ6: ACCR = %f" % accuracy)

َََQ6: ACCR = 0.870000


In [29]:
def summary(CNN):
    total_params = 0
    print("Layer-wise summary:")
    for i, layer in enumerate(CNN):
        layer_name = layer.__class__.__name__
        params = layer.params if hasattr(layer, 'params') else 0
        total_params += params
        print(f"Layer {i}: {layer_name} - Parameters: {params}")

    print("\nTotal trainable parameters:", total_params)
summary(CNN)

Layer-wise summary:
Layer 0: conv_layer - Parameters: 0
Layer 1: ReLU - Parameters: 0
Layer 2: pool_layer - Parameters: 0
Layer 3: conv_layer - Parameters: 0
Layer 4: ReLU - Parameters: 0
Layer 5: pool_layer - Parameters: 0
Layer 6: conv_layer - Parameters: 0
Layer 7: ReLU - Parameters: 0
Layer 8: pool_layer - Parameters: 0
Layer 9: reshape_layer - Parameters: 0
Layer 10: weights_layer - Parameters: 0
Layer 11: ReLU - Parameters: 0
Layer 12: dropout_layer - Parameters: 0
Layer 13: weights_layer - Parameters: 0
Layer 14: ReLU - Parameters: 0
Layer 15: dropout_layer - Parameters: 0
Layer 16: weights_layer - Parameters: 0

Total trainable parameters: 0


In [32]:
def summary(CNN):
    total_params = 0
    print("Layer-wise summary:")
    for i, layer in enumerate(CNN):
        layer_name = layer.__class__.__name__
        params = getattr(layer, 'params', 0)
        total_params += params
        print(f"Layer {i}: {layer_name} - Parameters: {params}")

    print("\nTotal trainable parameters:", total_params)


CNN = []

CNN.append(conv_layer(8, (3,3,3), lr))
CNN.append(ReLU())
CNN.append(pool_layer((2,2)))

CNN.append(conv_layer(16, (3,3,8) , lr))
CNN.append(ReLU())
CNN.append(pool_layer((2,2)))


CNN.append(conv_layer(32, (3,3,16), lr))
CNN.append(ReLU())
CNN.append(pool_layer((2,2)))


CNN.append(reshape_layer())


CNN.append(weights_layer(128,100,lr))
CNN.append(ReLU())
CNN.append(dropout_layer(0.5))
CNN.append(weights_layer(100,100,lr))
CNN.append(ReLU())
CNN.append(dropout_layer(0.5))
CNN.append(weights_layer(100,5,lr))


summary(CNN)


Layer-wise summary:
Layer 0: conv_layer - Parameters: 216
Layer 1: ReLU - Parameters: 0
Layer 2: pool_layer - Parameters: 0
Layer 3: conv_layer - Parameters: 1152
Layer 4: ReLU - Parameters: 0
Layer 5: pool_layer - Parameters: 0
Layer 6: conv_layer - Parameters: 4608
Layer 7: ReLU - Parameters: 0
Layer 8: pool_layer - Parameters: 0
Layer 9: reshape_layer - Parameters: 0
Layer 10: weights_layer - Parameters: 12800
Layer 11: ReLU - Parameters: 0
Layer 12: dropout_layer - Parameters: 0
Layer 13: weights_layer - Parameters: 10000
Layer 14: ReLU - Parameters: 0
Layer 15: dropout_layer - Parameters: 0
Layer 16: weights_layer - Parameters: 500

Total trainable parameters: 29276
